In [0]:
import numpy as np
import pandas as pd
import cv2 as cv
from time import time
from scipy.stats import randint as sp_randint
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
import glob
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from tempfile import mkdtemp
from shutil import rmtree
from sklearn.externals.joblib import Memory
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score 

In [ ]:
label_filepath = "../reflex.csv"

In [16]:
feature_number,turn,iteration = 3,0,0  #3 - strongBackground
global n_est,cvv,score,feature_number,r_state,scoringg

cutter = [10,10,10,13,11,22,21]
feature_names = ["Loop scatter","Background ring","Strong Background","Diffuse scatter","Artifact","Ice ring","Non-uniform detector"]
suffix = ["min" , "max", "var", "median", "mean","95th_percentile","5th_percentile"]

def init_data():
    for turn in range(5, len(suffix)):
        print("Suffix=",suffix[turn], "| Step",str(turn)+"/"+str(len(suffix)),"| Feature=",feature_names[feature_number-1])
        # files read and cut suffix
        path = "1dNew/"+str(suffix[turn])+"/*"
        file_names = [fn for fn in glob.glob(path)]
        join_attr = []
        join_attr.extend([fn[cutter[turn] :-4] for fn in file_names])
        
        # read of 1d vectors (index with cutted names - without unnecessary :\path letters)
        X = [cv.imread(name, 0)[10][:45] for name in file_names]
        X = pd.DataFrame(X, index=join_attr)

        # read Y values from excel file 
        training_data = pd.read_csv(label_filepath)
        y_names = training_data.iloc[:, 0].str.slice(7, -4).values
        y = training_data.iloc[:, feature_number].values
        y = pd.DataFrame(y, index=y_names)
        # make 3 Dataframes - X with values of vectors, y with score, and joined Xy
        X_joined = X.join(y, how="inner", rsuffix = "_labels")
        y = X_joined.loc[:, "0_labels"]
        X = X_joined.drop("0_labels", axis=1)
        
        X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.1, random_state=42)
        print("Number of elements connected with file reflex.csv: ", len(X_joined))
        print("Training set size: {} & Test set size: {}".format(len(X_train),len(X_test)))
        
        lightGBM(X_train,X_test,y_train,y_test)        
        
        
def lightGBM(X_train,X_test,y_train,y_test):        
        gbm = lgb.LGBMRegressor(objective='regression',
                        num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=20)
    
        gbm.fit(X_train, y_train,
                        eval_set=[(X_test, y_test)],
                        eval_metric='l1',
                        early_stopping_rounds=5)
        
        y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
        # eval
        print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)
        # feature importances
        print('Feature importances:', list(gbm.feature_importances_))
        # other scikit-learn modules
        estimator = lgb.LGBMRegressor(num_leaves=31)
        
        ### Grid_serach
        param_grid = {
        "max_depth": [3,4,5,6, None],
        "min_samples_split": [2,5],#[2, 3, 10, 20, 50, 100],
        "min_samples_leaf": [2,3],#[1, 3, 10,40,80],
        "bootstrap": [True, False],
        "criterion": ["gini", "entropy"],
        "n_estimators":[5,10,20,40,50]}
        
        gbm = GridSearchCV(estimator, param_grid)
        gbm.fit(X_train, y_train)
        print('Best parameters found by grid search are:', gbm.best_params_)
        
if __name__ == '__main__' :
        init_data()


Suffix= 95th_percentile | Step 5/7 | Feature= Strong Background


FileNotFoundError: ignored